In [16]:
import pandas as pd
import os

In [17]:
files=[file for dirpath,directory,file in os.walk('/home/payal/all_csv/')][0]
# files=[file for dirpath,directory,file in os.walk(r'/gdrive/My Drive/all_csv/')][0]

In [18]:
match_data=pd.DataFrame(data=None)
odi_scorecard=pd.DataFrame(data=None)
odi_info=pd.DataFrame(data=None)
ttwenty_info=pd.DataFrame(data=None)
ttwenty_scorecard=pd.DataFrame(data=None)

In [19]:
def rename_date_umpire(index_list):
    n=0
    for i in range(index_list.__len__()):
        if str.lower(index_list[i]).strip()=='date':
            index_list[i]+='_'+str(n)
            n+=1
    n=0
    for i in range(index_list.__len__()):
        if str.lower(index_list[i]).strip()=='umpire':
            index_list[i]+='_'+str(n)
            n+=1
    return index_list

In [20]:
def find_game(df_game,df_info):
    if 'series' in df_info.columns:
        if 'odi' in str.lower(df_info.iloc[0]['series']):
            return 'odi'
        if 't20i' in str.lower(df_info.iloc[0]['series']) or 't20' in str.lower(df_info.iloc[0]['series']) or 'indian premier league' in str.lower(df_info.iloc[0]['series']) :
            return 'twenty'
    if max(df_game['balls_bowl'])<=24:
        return 'twenty'
    if 24<max(df_game['balls_bowl'])<=60:
        return 'odi'
    return

In [21]:
def append_file(temp_df,temp_info_df,type_game):
    global odi_scorecard
    global ttwenty_scorecard
    global odi_info
    global ttwenty_info 
    if type_game=='odi':
        odi_scorecard=odi_scorecard.append(temp_df,ignore_index=True)
        odi_info=odi_info.append(temp_info_df,ignore_index=True)
    elif type_game=='twenty':
        ttwenty_scorecard=ttwenty_scorecard.append(temp_df,ignore_index=True)
        ttwenty_info=ttwenty_info.append(temp_info_df,ignore_index=True)

In [22]:
def get_extras_type(match_data):
    list_extras=[]
    for index,row in match_data.iterrows():
        ov=str(row['over'])
        if '.' in ov:
            ov=str(row['over']).split('.')
            ball_no=int(ov[1])
            over_no=int(ov[0])
        else:
            continue
        if row['extras']!=0:
            if row['runs']!=0:
                match_data.loc[index,'extras_type']='w'
            list_extras.append(index)
        if ball_no>6:
            if len(list_extras)>0:
                match_data.loc[list_extras.pop(-1),'extras_type']='w'
    for i in list_extras:
        match_data.loc[i,'extras_type']='b'
    return match_data

In [8]:
def prepare_scorecard(match_data):
    match_data=get_extras_type(match_data)
#     print(match_data[match_data['bowler']==	'Mashrafe Mortaza'])
    players=list((match_data['striker'].append(match_data['non-striker']).append(match_data['bowler'])).unique())
    #     to make 22 players if any player has not played
#     for i in range(len(players),22):
#         players.append('p_'+str(i))
#         fow
#     player_stats=['match_id','inning-batting','name','position','over-batsman','score','balls_bat','ones','twos','threes','fours','sixes','out','balls_bowl','maidens','runs_bowled','wickets','bowler_extras','wicket_score','wicket_overs','wicket_no','fow','fow_runs','fow_overs','fow_batsman','fow_bowler']
    player_stats=['match_id','inning-batting','name','position','over_batsman','score','balls_bat','dots','ones','twos','threes','fours','sixes','out','balls_bowl','maidens','runs_bowled','wickets','bowler_extras','wicket_score','wicket_overs','wicket_no','wicket_bowler']
    player_data={key:{key_type:0 for key_type in player_stats} for key in players}
    for p in players:
        player_data[p]['match_id']=match_data.loc[0,'file_no']
        player_data[p]['name']=p
    team_score=0
    balls=0
    pos=1
    inning=False
    w=1
    p_no=1
    w_no=1
    for index, row in match_data.iterrows():
        ov=str(row['over'])
        if '.' in ov:
            ov=ov.split('.')
            ball_no=int(ov[1])
            over_no=int(ov[0])
        else:
            continue
        if over_no>50:
            player_data=[value for key, value in player_data.items()]
            scorecard=pd.DataFrame(data=player_data)
            scorecard=scorecard[player_stats]
            return scorecard
        if ball_no==1 and over_no==0:
            pos=1
            w=1
            team_score=0
            w_no=1
            runs_over=0
            # if row['innings']!=1:
            #     p_no=12
        if row['runs']==1:
            player_data[row['striker']]['ones']+=1
        elif row['runs']==2:
            player_data[row['striker']]['twos']+=1
        elif row['runs']==3:
            player_data[row['striker']]['threes']+=1
        elif row['runs']==4:
            player_data[row['striker']]['fours']+=1
        elif row['runs']==6:
            player_data[row['striker']]['sixes']+=1
        elif row['extras']==0:
            player_data[row['striker']]['dots']+=1

        if player_data[row['striker']]['position']==0:
            player_data[row['striker']]['position']=pos
            player_data[row['striker']]['over_batsman']=row['over']
            pos+=1
        if player_data[row['non-striker']]['position']==0:
            player_data[row['non-striker']]['position']=pos
            player_data[row['non-striker']]['over_batsman']=row['over']
            pos+=1
    # wicket
#         print(row['out-player'])
        if not pd.isna(row['out-player']):
            player_data[row['out-player']]['out']=row['out']
#             fow
#             player_data[players[p_no-1]]['fow']=w
#             player_data[players[p_no-1]]['fow_runs']=team_score
#             player_data[players[p_no-1]]['fow_overs']=row['over']
#             player_data[players[p_no-1]]['fow_batsman']=row['out-player']
#             player_data[players[p_no-1]]['fow_bowler']=row['bowler']
            p_no+=1
            w+=1
            if row['out']!='run out':
                player_data[row['bowler']]['wickets']+=1
            player_data[row['out-player']]['wicket_score']=team_score
            player_data[row['out-player']]['wicket_overs']=row['over']
            player_data[row['out-player']]['wicket_no']=w_no
            player_data[row['out-player']]['wicket_bowler']=row['bowler']
            w_no+=1
        team_score+=row['runs']+row['extras']
        runs_over+=row['runs']
        if row['extras']!=0 and row['extras_type']=='w':
            runs_over+=1
            player_data[row['bowler']]['bowler_extras']+=1
            player_data[row['striker']]['score']+=row['extras']-1
            player_data[row['bowler']]['runs_bowled']+=row['extras']-1
        elif row['extras']!=0:
#             print(row)
            player_data[row['bowler']]['balls_bowl']+=1
            player_data[row['striker']]['balls_bat']+=1
        else:
            player_data[row['striker']]['balls_bat']+=1
            player_data[row['bowler']]['balls_bowl']+=1
            player_data[row['bowler']]['runs_bowled']+=row['runs']
        player_data[row['striker']]['score']+=row['runs']
        if ball_no>=6:
            if ball_no==6 and runs_over==0:
                player_data[row['bowler']]['maidens']+=1
            runs_over=0
        player_data[row['striker']]['inning-batting']=row['innings']
        if row['innings']==1:
            player_data[row['bowler']]['inning-batting']=2
        else:
            player_data[row['bowler']]['inning-batting']=1
#     print(player_data)
    player_data=[value for key, value in player_data.items()]
    scorecard=pd.DataFrame(data=player_data)
    scorecard=scorecard[player_stats]
    return scorecard

In [9]:
i=0
for file in files[:]:
    print(i,'--',file)
    i+=1
    count=0
    df_index=[]
    df_row=[]
    add=r'/home/payal/all_csv/'+file
#     add=r'/gdrive/My Drive/all_csv/'+file
    df_index=['file_no']
    file_no=file.split('.')[0]
    df_row=[file_no]
    with open(add) as f:
        new_f=f.readlines()
        for line in new_f:
            if 'version' in line:
                count+=1
            elif 'info' in line:
                count+=1
                line=line.strip().split(',')
                df_index.append(line[1])
                df_row.append(line[2])
            else:
                df_index=rename_date_umpire(df_index)
                df_dic=dict(zip(df_index,df_row),index=[0])
                temp_info_df=pd.DataFrame(df_dic)
#                 df_info=df_info.append(temp_info_df,ignore_index=True)
                # gender=df_info['gender'].iloc[0]
                # gender=str.lower(gender.strip())
                break         
    temp_df=pd.read_csv(add,skiprows=count,names=[0,'innings','over','batting-team','striker','non-striker','bowler','runs','extras','out','out-player'])
    temp_df=temp_df.drop([0],axis=1)
    temp_df['file_no']=[file_no]*(temp_df.shape[0])
    temp_df['over_no']=temp_df['over'].apply(lambda x: str(x).split('.')[0])
    temp_df=temp_df[temp_df['over_no']=='0']
#     print(temp_df)
    temp_sc=prepare_scorecard(temp_df)
    # print(temp_sc)
#     append_file(temp_df,gender,type_game) 
    append_file(temp_sc,temp_info_df,find_game(temp_sc,temp_info_df))

0 -- 660131.csv
1 -- 951419.csv


/home/payal/anaconda3/envs/home/lib/python3.7/site-packages/pandas/core/frame.py:7116: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2 -- 736519.csv
3 -- 951411.csv
4 -- 501257.csv
5 -- 636165.csv
6 -- 913617.csv
7 -- 335353.csv
8 -- 1089522.csv
9 -- 1023617.csv
10 -- 433606.csv
11 -- 966373.csv
12 -- 548344.csv
13 -- 336021.csv
14 -- 587473.csv
15 -- 1019979.csv
16 -- 428751.csv
17 -- 447539.csv
18 -- 1082599.csv
19 -- 548335.csv
20 -- 913655.csv
21 -- 946549.csv
22 -- 446962.csv
23 -- 1020029.csv
24 -- 426720.csv
25 -- 258464.csv
26 -- 257772.csv
27 -- 785963.csv
28 -- 1062580.csv
29 -- 656487.csv
30 -- 249757.csv
31 -- 913657.csv
32 -- 439142.csv
33 -- 733987.csv
34 -- 951309.csv
35 -- 518947.csv
36 -- 915775.csv
37 -- 1062505.csv
38 -- 756801.csv
39 -- 433594.csv
40 -- 667895.csv
41 -- 1034829.csv
42 -- 251488.csv
43 -- 593722.csv
44 -- 534205.csv
45 -- 238200.csv
46 -- 729299.csv
47 -- 524942.csv
48 -- 654067.csv
49 -- 1144496.csv
50 -- 392194.csv
51 -- 531983.csv
52 -- 682991.csv
53 -- 366629.csv
54 -- 434256.csv
55 -- 352662.csv
56 -- 1077955.csv
57 -- 1178412.csv
58 -- 1085970.csv
59 -- 548330.csv
60 -- 7279

463 -- 749551.csv
464 -- 749561.csv
465 -- 798619.csv
466 -- 777995.csv
467 -- 336037.csv
468 -- 1190608.csv
469 -- 645641.csv
470 -- 589308.csv
471 -- 298808.csv
472 -- 777999.csv
473 -- 291366.csv
474 -- 660121.csv
475 -- 1073420.csv
476 -- 521218.csv
477 -- 319146.csv
478 -- 785983.csv
479 -- 598008.csv
480 -- 291218.csv
481 -- 980937.csv
482 -- 1136598.csv
483 -- 287858.csv
484 -- 1043955.csv
485 -- 501259.csv
486 -- 656477.csv
487 -- 518966.csv
488 -- 439144.csv
489 -- 296917.csv
490 -- 275789.csv
491 -- 733973.csv
492 -- 573022.csv
493 -- 661697.csv
494 -- 902637.csv
495 -- 951359.csv
496 -- 1136590.csv
497 -- 299006.csv
498 -- 433582.csv
499 -- 535795.csv
500 -- 660221.csv
501 -- 343731.csv
502 -- 1178407.csv
503 -- 535997.csv
504 -- 392201.csv
505 -- 1029819.csv
506 -- 501228.csv
507 -- 1089780.csv
508 -- 628096.csv
509 -- 518969.csv
510 -- 661685.csv
511 -- 953345.csv
512 -- 287853.csv
513 -- 419107.csv
514 -- 951367.csv
515 -- 531632.csv
516 -- 661681.csv
517 -- 562441.csv
51

916 -- 335989.csv
917 -- 593728.csv
918 -- 430887.csv
919 -- 1144522.csv
920 -- 1034821.csv
921 -- 255954.csv
922 -- 757511.csv
923 -- 951349.csv
924 -- 829751.csv
925 -- 401076.csv
926 -- 1178428.csv
927 -- 897697.csv
928 -- 1152517.csv
929 -- 1073426.csv
930 -- 297796.csv
931 -- 598045.csv
932 -- 247504.csv
933 -- 656489.csv
934 -- 1136601.csv
935 -- 1030227.csv
936 -- 461571.csv
937 -- 729289.csv
938 -- 392207.csv
939 -- 452148.csv
940 -- 351693.csv
941 -- 1082592.csv
942 -- 875547.csv
943 -- 456665.csv
944 -- 293072.csv
945 -- 1152515.csv
946 -- 564784.csv
947 -- 766931.csv
948 -- 966759.csv
949 -- 361045.csv
950 -- 249233.csv
951 -- 602473.csv
952 -- 800463.csv
953 -- 548324.csv
954 -- 406195.csv
955 -- 973777.csv
956 -- 913615.csv
957 -- 1020033.csv
958 -- 749773.csv
959 -- 473920.csv
960 -- 1050225.csv
961 -- 913455.csv
962 -- 676525.csv
963 -- 1175371.csv
964 -- 433601.csv
965 -- 1085961.csv
966 -- 514027.csv
967 -- 287873.csv
968 -- 535797.csv
969 -- 598069.csv
970 -- 654063.c

...

2206 -- 534225.csv
2207 -- 299009.csv
2208 -- 980925.csv
2209 -- 501264.csv
2210 -- 654073.csv
2211 -- 1022599.csv
2212 -- 734039.csv
2213 -- 875551.csv
2214 -- 520602.csv
2215 -- 325572.csv
2216 -- 860281.csv
2217 -- 576409.csv
2218 -- 693433.csv
2219 -- 426404.csv
2220 -- 1114877.csv
2221 -- 936127.csv
2222 -- 1114884.csv
2223 -- 777997.csv
2224 -- 1068740.csv
2225 -- 546477.csv
2226 -- 439154.csv
2227 -- 668959.csv
2228 -- 690351.csv
2229 -- 829791.csv
2230 -- 426395.csv
2231 -- 249747.csv
2232 -- 1136597.csv
2233 -- 1085954.csv
2234 -- 940413.csv
2235 -- 885971.csv
2236 -- 924637.csv
2237 -- 654087.csv
2238 -- 778027.csv
2239 -- 434257.csv
2240 -- 533306.csv
2241 -- 238178.csv
2242 -- 336040.csv
2243 -- 875471.csv
2244 -- 535999.csv
2245 -- 388993.csv
2246 -- 660145.csv
2247 -- 1023601.csv
2248 -- 640935.csv
2249 -- 656457.csv
2250 -- 548371.csv
2251 -- 251496.csv
2252 -- 930575.csv
2253 -- 1168522.csv
2254 -- 501215.csv
2255 -- 287857.csv
2256 -- 897711.csv
2257 -- 433589.csv
2258

2633 -- 518970.csv
2634 -- 452154.csv
2635 -- 860269.csv
2636 -- 951313.csv
2637 -- 336024.csv
2638 -- 756803.csv
2639 -- 749547.csv
2640 -- 733991.csv
2641 -- 914205.csv
2642 -- 300487.csv
2643 -- 1034811.csv
2644 -- 518957.csv
2645 -- 829785.csv
2646 -- 1168521.csv
2647 -- 598000.csv
2648 -- 936167.csv
2649 -- 623576.csv
2650 -- 540172.csv
2651 -- 811125.csv
2652 -- 1019973.csv
2653 -- 1089779.csv
2654 -- 564786.csv
2655 -- 540169.csv
2656 -- 412677.csv
2657 -- 946545.csv
2658 -- 1082595.csv
2659 -- 598043.csv
2660 -- 392186.csv
2661 -- 501208.csv
2662 -- 913451.csv
2663 -- 350480.csv
2664 -- 501235.csv
2665 -- 531637.csv
2666 -- 298804.csv
2667 -- 386531.csv
2668 -- 598031.csv
2669 -- 523735.csv
2670 -- 636163.csv
2671 -- 534230.csv
2672 -- 980951.csv
2673 -- 829803.csv
2674 -- 582192.csv
2675 -- 428750.csv
2676 -- 954743.csv
2677 -- 1144530.csv
2678 -- 962017.csv
2679 -- 1085971.csv
2680 -- 403369.csv
2681 -- 392205.csv
2682 -- 356014.csv
2683 -- 648677.csv
2684 -- 1152562.csv
2685

3060 -- 829759.csv
3061 -- 1041615.csv
3062 -- 902639.csv
3063 -- 980999.csv
3064 -- 1114860.csv
3065 -- 415273.csv
3066 -- 296907.csv
3067 -- 1152519.csv
3068 -- 648653.csv
3069 -- 366708.csv
3070 -- 564782.csv
3071 -- 291360.csv
3072 -- 1085947.csv
3073 -- 963701.csv
3074 -- 527017.csv
3075 -- 778023.csv
3076 -- 426405.csv
3077 -- 1089520.csv
3078 -- 523733.csv
3079 -- 501224.csv
3080 -- 868729.csv
3081 -- 756735.csv
3082 -- 1031437.csv
3083 -- 875459.csv
3084 -- 392212.csv
3085 -- 848841.csv
3086 -- 1007657.csv
3087 -- 392190.csv
3088 -- 1136603.csv
3089 -- 548308.csv
3090 -- 356008.csv
3091 -- 1082610.csv
3092 -- 587470.csv
3093 -- 810833.csv
3094 -- 656461.csv
3095 -- 571947.csv
3096 -- 952197.csv
3097 -- 1027315.csv
3098 -- 1114893.csv
3099 -- 392191.csv
3100 -- 598070.csv
3101 -- 434258.csv
3102 -- 628086.csv
3103 -- 627009.csv
3104 -- 345973.csv
3105 -- 415278.csv
3106 -- 722391.csv
3107 -- 897725.csv
3108 -- 1089530.csv
3109 -- 247476.csv
3110 -- 297801.csv
3111 -- 750673.csv


3488 -- 953105.csv
3489 -- 889463.csv
3490 -- 829809.csv
3491 -- 551593.csv
3492 -- 566927.csv
3493 -- 881737.csv
3494 -- 518960.csv
3495 -- 531985.csv
3496 -- 1020043.csv
3497 -- 654059.csv
3498 -- 800479.csv
3499 -- 1144525.csv
3500 -- 573011.csv
3501 -- 562446.csv
3502 -- 875513.csv
3503 -- 1144508.csv
3504 -- 907381.csv
3505 -- 1136615.csv
3506 -- 352664.csv
3507 -- 531986.csv
3508 -- 593986.csv
3509 -- 656485.csv
3510 -- 392203.csv
3511 -- 259793.csv
3512 -- 597926.csv
3513 -- 1050611.csv
3514 -- 392235.csv
3515 -- 682905.csv
3516 -- 1019983.csv
3517 -- 895813.csv
3518 -- 319133.csv
3519 -- 463151.csv
3520 -- 1082612.csv
3521 -- 351680.csv
3522 -- 656417.csv
3523 -- 1074957.csv
3524 -- 456670.csv
3525 -- 631138.csv
3526 -- 736451.csv
3527 -- 238175.csv
3528 -- 422629.csv
3529 -- 757509.csv
3530 -- 1114897.csv
3531 -- 540171.csv
3532 -- 875511.csv
3533 -- 567367.csv
3534 -- 522250.csv
3535 -- 441828.csv
3536 -- 1114879.csv
3537 -- 352665.csv
3538 -- 291371.csv
3539 -- 571149.csv
35

3915 -- 858491.csv
3916 -- 598063.csv
3917 -- 754749.csv
3918 -- 734029.csv
3919 -- 426391.csv
3920 -- 530425.csv
3921 -- 894291.csv
3922 -- 644945.csv
3923 -- 226383.csv
3924 -- 1050223.csv
3925 -- 1181768.csv
3926 -- 366626.csv
3927 -- 598019.csv
3928 -- 412682.csv
3929 -- 875507.csv
3930 -- 247464.csv
3931 -- 598067.csv
3932 -- 736517.csv
3933 -- 489220.csv
3934 -- 446956.csv
3935 -- 455235.csv
3936 -- 1175362.csv
3937 -- 756039.csv
3938 -- 660187.csv
3939 -- 1034827.csv
3940 -- 683011.csv
3941 -- 534209.csv
3942 -- 433563.csv
3943 -- 914239.csv
3944 -- 392197.csv
3945 -- 656423.csv
3946 -- 902651.csv
3947 -- 452146.csv
3948 -- 1089472.csv
3949 -- 980969.csv
3950 -- 952193.csv
3951 -- 535799.csv
3952 -- 257470.csv
3953 -- 875533.csv
3954 -- 501243.csv
3955 -- 980929.csv
3956 -- 524915.csv
3957 -- 533291.csv
3958 -- 298798.csv
3959 -- 501262.csv
3960 -- 433558.csv
3961 -- 403375.csv
3962 -- 565814.csv
3963 -- 903601.csv
3964 -- 729295.csv
3965 -- 319142.csv
3966 -- 566946.csv
3967 --

In [25]:
odi_info.columns

Index(['city', 'competition', 'date_0', 'date_1', 'file_no', 'gender', 'index',
       'match_number', 'match_referee', 'method', 'neutralvenue', 'outcome',
       'player_of_match', 'reserve_umpire', 'season', 'series', 'team',
       'toss_decision', 'toss_winner', 'tv_umpire', 'umpire_0', 'umpire_1',
       'venue', 'winner', 'winner_runs', 'winner_wickets'],
      dtype='object')

In [27]:
odi_scorecard.head()

,match_id,inning-batting,name,position,over_batsman,score,balls_bat,dots,ones,twos,...,out,balls_bowl,maidens,runs_bowled,wickets,bowler_extras,wicket_score,wicket_overs,wicket_no,wicket_bowler
0,587473,1,Tamim Iqbal,1,0.1,1,5,4,1,0,...,0,0,0,0,0,0,0,0.0,0,0
1,587473,1,Anamul Haque,2,0.1,4,1,0,0,0,...,0,0,0,0,0,0,0,0.0,0,0
2,587473,2,CH Gayle,1,0.1,0,6,6,0,0,...,0,0,0,0,0,0,0,0.0,0,0
3,587473,0,KOA Powell,2,0.1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0,0
4,587473,2,R Rampaul,0,0.0,0,0,0,0,0,...,0,6,0,5,0,0,0,0.0,0,0


In [12]:
odi_info.shape

(883, 26)

In [15]:
odi_info.head()

,city,competition,date_0,date_1,file_no,gender,index,match_number,match_referee,method,...,team,toss_decision,toss_winner,tv_umpire,umpire_0,umpire_1,venue,winner,winner_runs,winner_wickets
0,Mirpur,NaN,2012/12/05,NaN,587473,male,0,3,AJ Pycroft,NaN,...,West Indies,field,West Indies,Anisur Rahman,Enamul Haque,REJ Martinesz,Shere Bangla National Stadium,West Indies,NaN,4
1,London,NaN,2016/08/27,NaN,913655,male,0,2,JJ Crowe,NaN,...,Pakistan,bat,Pakistan,SD Fry,M Erasmus,RT Robinson,Lord's,England,NaN,4
2,Taunton,"""ICC Women's Championship",2016/06/27,NaN,946549,female,0,3,DT Jukes,NaN,...,Pakistan,bat,England,AG Wharf,DJ Millns,MJ Saggers,County Ground,England,202,NaN
3,,NaN,2011/01/16,NaN,446962,male,0,1,RS Madugalle,NaN,...,England,bat,England,AL Hill,BF Bowden,BNJ Oxenford,Melbourne Cricket Ground,Australia,NaN,6
4,Abu Dhabi,NaN,2009/11/03,NaN,426720,male,0,1,AJ Pycroft,NaN,...,Pakistan,bat,Pakistan,Zameer Haider,Asad Rauf,BNJ Oxenford,Sheikh Zayed Stadium,Pakistan,138,NaN


In [28]:
odi_info.to_csv('./odi_info.csv',index=False)

odi_scorecard.to_csv('./odi_scorecard.csv',index=False)


In [29]:
!pwd

/home/payal/Downloads
